# Final Project

## Topological Features in Machine Learning
Dean Stuart \
Math 254\
Spring 2022\
Final Project

For my project I want to show how using topological features and tools can improve the results of a machine learning algorithm. I am soliving a problem involiving classifying handwritten digits based on the image of them alone. The data is from Kaggle and has 42000 images of digits 0-9. 
More indepth descussion of the topology topics follow between code chunks below.

With this project I intend to enter the results into the Kaggle competition.

#### Imports

In [97]:
# General Imports
import numpy as np
import pandas as pd
import csv

# TDA Imports
from gtda.plotting import plot_heatmap
from gtda.images import Binarizer, DensityFiltration, RadialFiltration
from gtda.homology import CubicalPersistence
from gtda.diagrams import Amplitude, PersistenceEntropy, Scaler

# Sklearn Machine Learning Imports
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline, make_pipeline, make_union


The first step is to import my data. The csv 'digits.csv' is from Kaggle and is 42001 rows with 785 columns with a header row.
I removed the header row and seperate the 785 columns into y, the target Series which is the correct digit label of each observation, and X, the DataFrame of the 28*28=784 rows which represent each pixel in the image of the handwritten digit. The shape of each is outputted below.

In [6]:
my_data = np.genfromtxt('digits.csv', delimiter=',')
data = np.delete(my_data,0,0)

y = pd.Series(data[:,0])
X = pd.DataFrame(data[:,1:])

In [7]:
print(f"X shape: {X.shape}, y shape: {y.shape}")

X shape: (42000, 784), y shape: (42000,)


### Example on One Image

Here I find the first observation that is labeled '8' and print the image out as an example of what one of these handwritten images look like.\
This image is the running example and it was chosen because an 8 typically has 2 loops in it and plent of connected spaces making it a good image to pick and look at.

In [49]:
# Reshape to (n_samples, n_pixels_x, n_pixels_y)
im8_idx = np.flatnonzero(y == 8)[0]
img8 = X.values[im8_idx].reshape(28, 28)
plot_heatmap(img8)

![title](figs/1.png)

Reshape the X DataFrame so each observation is 28x28 instead of  1x784

In [44]:
X_reshaped = X.values.reshape((-1, 28, 28))

Here is a binary version of the same image as above. The image above has pixels ranging in value from 0-255 which is common way to represent pixel color. It is necessary for the topological features to look at pixels as either active or not. This is done by selecting pixels that are determined dark enough by the Binarizer threshold paramater.\
The parameter was handpicked at .7 so any pixel with a value of >178. 0.7 was the choice because it gave a cleaner look for the image. Later on, 0.6 is used in the machine learning algorithm to allow the topological features to see more pixels, but still ignore the noise. 

In [50]:
im8 = X_reshaped[im8_idx][None, :, :]

binarizer = Binarizer(threshold=.7)
im8_binarized = binarizer.fit_transform(im8)

binarizer.plot(im8_binarized)

![title](figs/2.png)

With this binary image, we can look at this image through differnet filters. This example uses the Density Filtraton, Radial Filtration will also be used later and will be discussed then too. Density Filtraton takes each pixel and builds and epsilon ball around it's center with the radius of that ball being equal to the DensityFiltration radius parameter. 1 is used in this case so only looking at adjacent pixels. The value of that pixel is then set to the number of pixels in said epsilon ball that are active. You can see the two holes in the middle of the graph as well as connected compnents with all adjecent pixels active. 

In [71]:
density_filtration = DensityFiltration(radius=1,metric='euclidean')
im8_filtration = density_filtration.fit_transform(im8_binarized)

density_filtration.plot(im8_filtration, colorscale="earth")

![title](figs/3.png)

With this filtration we can build a Persistence Diagram which isn't directly used in the machine learning algorthim but can be a useful visualization of the topological features. This graph tells us the persistence of homology classes at different scales of the radius of the ball. These homology classes are born and die at different points on the scale and the further they are from the center line, the more persistant they are. The first dimension of the topological features (H0 seen with red dots) are the connected components and the second dimension (H1 seen with blue dots) are the holes in the graph. 

In [72]:
cubical_persistence = CubicalPersistence(n_jobs=-1)
im8_cubical = cubical_persistence.fit_transform(im8_filtration)

cubical_persistence.plot(im8_cubical,homology_dimensions=None)

![title](figs/4.png)

As visible in the above graph, there are several holes with 2 being more persistant than the others, representing the classic two loops in an 8. There is also one connected compenent more persistant than the other because of the large part of the graph at the top of the 8 that has lots of activated pixels. These features of homology classes are what will be feed to the machine learning algortithm, rather than the raw data of the pixel values. 

### Building a Pipeline

Using sklearn's Pipeline, it is possible to combine all of the above work into one function that can be used on any image in the data set.\
This pipeline is just used as an example and has all the same parameters and functions as the running example.

The final step, Amplitude vectorizes the graph, finding the amplitude of the persistance of the features of the Persistance Diagram. This vector is what is actually feed to the machine learning algorithm. The metric for calculating amplitude will be explained later. This gives a vector of size 2 representing both dimensions show in the Diagram before.

In [57]:
steps = [
    ("binarizer", Binarizer(threshold=0.7)),
    ("filtration", DensityFiltration(radius=1,metric='euclidean')),
    ("diagram", CubicalPersistence(n_jobs=-1)),
    ("amplitude", Amplitude(metric="wasserstein", metric_params={}))
]

heat_pipeline = Pipeline(steps)

Using the pipeline on the running example as well an image of a 9 to show the use. 

In [58]:
im8_pipeline = heat_pipeline.fit_transform(im8)
im8_pipeline

array([[2.23606798, 3.04138127]])

In [74]:
im9_idx = np.flatnonzero(y == 9)[0]
im9 = X_reshaped[im9_idx][None, :, :]

im9_pipeline = heat_pipeline.fit_transform(im9)
im9_pipeline


array([[1.        , 3.39116499]])

### Building a Machine Learning Solution

As typical, we will split the data into two sets, one for training the model and one for testing it.\
For sake of example, only around 30% of the data is used but results were strong enough that the time saved in testing this with only 10000 was worth giving up the other 30000.

In [75]:
X_train, X_test, y_train, y_test = train_test_split(
    X_reshaped, y, stratify=y, random_state=254, train_size=10000, test_size=2500
)

print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

X_train shape: (10000, 28, 28), y_train shape: (10000,)
X_test shape: (2500, 28, 28), y_test shape: (2500,)


#### Transformation Pipeline

This is the full pipeline that will be used to create the vectors of features that will be used.

The same DensityFiltration as above is used here with different radii ranging from 1-6.\
Another filter, RadialFiltration, is also used to create the scale image that will be used to calcuate the persistant features. This function takes a given center and a radius, and builds a ball around that center, here radius is None so all points on the graph are in the ball. The value of each pixel is the distance away from the center the point is using the given metric if the pixel is active. If not then the value is set to the maximum distance found plus 1.

The diagram is then created with the same steps as before but with the Binarizer threshold set to 0.6 to allow more pixels to be active\
Then just as before the amplitude of the diagram is calcuated this time with several different metrics and parameter of those metrics. In addition to the amplitude of the featrues, the entropy of the features is also calculated. This is the entropy of the lifetime of the homology classes.

All put togther, these create the topological features to be used in machine learning. After transforming the training data with this pipeline, these are the 210 features seen in the size of the X_train_tda set. 

In [87]:
# DensityFiltration radii
radius_list = [1,2,3,4,5,6]
# RadialFiltration centers
center_list = [
    [13, 6],
    [6, 13],
    [13, 13],
    [20, 13],
    [13, 20],
    [6, 6],
    [6, 20],
    [20, 6],
    [20, 20],
]

# list of all the filtration transformers
filtration_list = (
    [DensityFiltration(radius=r, metric='euclidean') for r in radius_list] + 
    [RadialFiltration(center=np.array(center), n_jobs=-1, metric='euclidean') for center in center_list]
)

# diagram creation pipeline
diagram_steps = [
    [
        Binarizer(threshold=0.6, n_jobs=-1),
        filtration,
        CubicalPersistence(n_jobs=-1),
        Scaler(n_jobs=-1),
    ]
    for filtration in filtration_list
]

# list all metrics to use for diagram amplitudes
metric_list = [
    {"metric": "wasserstein", "metric_params": {"p": 1}},
    {"metric": "wasserstein", "metric_params": {"p": 2}},
    {"metric": "landscape", "metric_params": {"p": 1, "n_layers": 1, "n_bins": 100}},
    {"metric": "landscape", "metric_params": {"p": 1, "n_layers": 2, "n_bins": 100}},
    {"metric": "landscape", "metric_params": {"p": 2, "n_layers": 1, "n_bins": 100}},
    {"metric": "landscape", "metric_params": {"p": 2, "n_layers": 2, "n_bins": 100}},
]

feature_union = make_union(
    *[PersistenceEntropy(nan_fill_value=-1)]
    + [Amplitude(**metric, n_jobs=-1) for metric in metric_list]
)

tda_union = make_union(
    *[make_pipeline(*diagram_step, feature_union) for diagram_step in diagram_steps],
    n_jobs=-1
)

In [88]:
X_train_tda = tda_union.fit_transform(X_train)
X_train_tda.shape

(10000, 210)

#### Using the Topological Features

Here the transformed data is feed to a simple Multi-layer Perceptron classifier. This neural network was selected because it is was less complex than a convolutional neural network meaning it is more effecient to run.\
Convolutional neural network is what would typically be used in a computer vision problem such as this, if topology wasn't used to transform the data. 

The test data is then transformed using the same pipeline and is used to score the classifier trained on the topological features. Score is seen below.

In [89]:
clf = MLPClassifier(activation='relu', hidden_layer_sizes=(5), solver='adam', alpha=1e-5,random_state=254)
clf.fit(X_train_tda, y_train)

X_test_tda = tda_union.transform(X_test)
clf.score(X_test_tda, y_test)

C:\Users\deanp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



0.8852

Here the same data is split again using the exact same split and then feed into the exact same Multilayer Perceptron classifier.

The model is then scored and the score can be seen below.

In [63]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(
    X, y, stratify=y, random_state=254, train_size=10000, test_size=2500
)
clf2 = MLPClassifier(activation='relu', hidden_layer_sizes=(5), solver='adam', alpha=1e-5,random_state=254)
clf2.fit(X_train2, y_train2)
clf2.score(X_test2, y_test2)

0.1116

### Conclusion

In this example, using topological features definitely improved the model over using raw data. This can be seen with the accuracy of 0.8852 vs 0.1116. While it is possible to build a more accurate classifier using the raw data, a much more complex model would be need which is computationally expensive. Using the topological transformations allows a user to save on computer power when doing machine learning. 

### Create Kaggle Submission

Repeat transformation above using test.csv then predict using the model.

In [93]:
kaggle_data = np.genfromtxt('test.csv', delimiter=',')
test_data = np.delete(kaggle_data,0,0)

In [94]:
test_reshaped = pd.DataFrame(test_data).values.reshape((-1, 28, 28))

In [95]:
test_tda = tda_union.fit_transform(test_reshaped)

In [96]:
pred = clf.predict(test_tda)

In [102]:
with open('submission.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(['ImageId','Label'])
    for i in range(len(pred)):
        spamwriter.writerow([i+1,int(pred[i])])

I got a score of 0.87432 in the Kaggle Competion 

### Resources

https://towardsdatascience.com/getting-started-with-giotto-learn-a-python-library-for-topological-machine-learning-451d88d2c4bc\
https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html\
https://www.kaggle.com/competitions/digit-recognizer/data\
https://www.frontiersin.org/articles/10.3389/frai.2021.681108/full#h4